# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np 
import pandas as pd 
import requests 
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('data/sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
api_token = 'Tpk_059b97af715d417d9f49f50b51b1c448'

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol='AAPL'
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={api_token}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 129914239, 'calculationPrice': 'close', 'change': -1.43, 'changePercent': -0.00847, 'close': 0, 'closeSource': 'alofcfii', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 170.09, 'iexCloseTime': 1716558835956, 'iexLastUpdated': 1695087787802, 'iexMarketPercent': 16.19256000536048, 'iexOpen': 168.16, 'iexOpenTime': 1655235912251, 'iexRealtimePrice': 173.58, 'iexRealtimeSize': 104, 'iexVolume': 1356347, 'lastTradeTime': 1666533315352, 'latestPrice': 177.41, 'latestSource': 'Close', 'latestTime': 'December 20, 2021', 'latestUpdate': 1653944585417, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2879400019118, 'oddLotDelayedPrice': None, 'odd

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
columns = ['Ticker','Stock Price','Market Capitalization','Number of Shares to Buy']
df = pd.DataFrame(columns=columns)
df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [7]:
df.append(
pd.Series([
    symbol,
    price,
    market_cap,
    'N/A'
],index = columns
), ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,177.41,2879400019118,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [12]:
# df = pd.DataFrame(columns=columns)
# for stock in stocks['Ticker']:
#     api_url=f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={api_token}'
#     data = requests.get(api_url).json()
#     df=df.append(pd.Series([
#     stock,
#     data['latestPrice'],
#     data['marketCap'],
#     'N/A'
# ],index = columns
# ), ignore_index=True
# )
# df.to_csv('api_data',as_index=False)
df=pd.read_csv('api_data')

In [15]:
df=df.drop("Unnamed: 0",axis=1)

In [16]:
df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,153.52,45702549320,NaN
1,AAL,17.70,11351406803,NaN
2,AAP,235.82,14710957025,NaN
3,AAPL,171.69,2794818217432,NaN
4,ABBV,133.89,240907440683,NaN
...,...,...,...,...
500,YUM,134.94,38865470745,NaN
501,ZBH,119.19,25739300826,NaN
502,ZBRA,591.04,31663705531,NaN
503,ZION,59.80,9455386750,NaN


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [17]:
def chunks(lst,n):
    for i in range(0, len(lst),n):
        yield lst[i:i+n]

In [31]:
symbol_groups  =  list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
final_dataframe = pd.DataFrame(columns=columns)

for symbol_string in symbol_strings:
    batch_api_call_url=f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={api_token}"
    data=requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(pd.Series([
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['marketCap'],
            'N/A'
        ],index=columns),
        ignore_index=True)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,154.22,45975969296,N/A
1,AAL,17.70,11760488464,N/A
2,AAP,237.80,14311367534,N/A
3,AAPL,175.26,2867250219136,N/A
4,ABBV,136.33,238521565496,N/A
...,...,...,...,...
500,YUM,136.24,40471651901,N/A
501,ZBH,121.16,24937945772,N/A
502,ZBRA,576.39,31993799075,N/A
503,ZION,61.20,9726256473,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [33]:
inp=input("enter value of your portfolio: ")
try:
    val = float(inp)
except ValueError:
    print("enter a number please!")
    inp=input("enter value of your portfolio: ")
    val = float(inp)

enter value of your portfolio:  1000000


In [37]:
position_size= val/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,"Number of Shares to Buy"] = math.floor(position_size/final_dataframe.loc[i, "Stock Price"])


In [38]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,154.22,45975969296,12
1,AAL,17.70,11760488464,111
2,AAP,237.80,14311367534,8
3,AAPL,175.26,2867250219136,11
4,ABBV,136.33,238521565496,14
...,...,...,...,...
500,YUM,136.24,40471651901,14
501,ZBH,121.16,24937945772,16
502,ZBRA,576.39,31993799075,3
503,ZION,61.20,9726256473,32


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [40]:
writer= pd.ExcelWriter('recommended trades.xlsx',engine='xlsxwriter')
final_dataframe.to_excel(writer, "Recommeneded Trades",index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: